In [1]:
!pip install -q transformers sentencepiece bs4


In [2]:
# Imports
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
from IPython.display import Markdown, display

# Scrape a webpage's text
def get_page_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    paragraphs = soup.find_all("p")
    return "\n".join([p.get_text() for p in paragraphs])

# Load summarizer pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Summarize webpage
def summarize_webpage(url):
    page_text = get_page_text(url)

    # BART has token limit ~1024, so truncate long texts
    chunks = [page_text[i:i+1024] for i in range(0, len(page_text), 1024)]

    summary = []
    for chunk in chunks[:3]:  # Avoid too many chunks (limit for Colab)
        result = summarizer(chunk, max_length=150, min_length=40, do_sample=False)
        summary.append(result[0]['summary_text'])

    return "\n\n".join(summary)

# Example URL
url = "https://en.wikipedia.org/wiki/Machine_learning"
summary = summarize_webpage(url)

# Display in notebook
display(Markdown(f"### 📝 Summary of Web Page:\n\n{summary}"))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


### 📝 Summary of Web Page:

Machine learning is the study of algorithms that can learn from data and generalise to unseen data. ML has applications in natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.

The term machine learning was coined in 1959 by IBM employee Arthur Samuel. The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers. The history of machine learning roots back to decades of human desire and effort to study human cognitive processes.

By the early 1960s, an experimental "learning machine" with punched tape memory, called Cybertron, had been developed by Raytheon Company to analyse sonar signals, electrocardiograms, and speech patterns. A representative book on research into machine learning during the 1960s was Nilsson's book on Learning Machines. Interest related to pattern recognition continued into the 1970s, as described by Duda and Hart in 1973.

In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

def ask_flan(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, max_length=200)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example
print(ask_flan("Summarize this: Good Morning!Today news:"))

Today's Top News:
